# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

In [2]:
m.release.aliases('demand')

In [3]:
m.a = Resource(consume = 10, consume_leq = 15, consume_geq = 4, demand_nominal = 180, release_geq = [20, 30, 40, 50])

2025-10-21 11:06:26,185 [INFO] General Resource Balance for a in (l, y): initializing constraint, adding consume(a, l, y)
2025-10-21 11:06:26,185 [INFO] ✔ Completed in 0.00016617774963378906 seconds
2025-10-21 11:06:26,186 [INFO] Binding consume in domain (a, l, y)
2025-10-21 11:06:26,186 [INFO] ✔ Completed in 9.417533874511719e-05 seconds
2025-10-21 11:06:26,186 [INFO] Binding consume in domain (a, l, y)
2025-10-21 11:06:26,187 [INFO] ✔ Completed in 7.915496826171875e-05 seconds
2025-10-21 11:06:26,187 [INFO] Binding consume in domain (a, l, y)
2025-10-21 11:06:26,187 [INFO] ✔ Completed in 0.00010418891906738281 seconds
2025-10-21 11:06:26,188 [INFO] Mapping release: (a, l, q) → (a, l, y)
2025-10-21 11:06:26,188 [INFO] ✔ Completed in 0.000s
2025-10-21 11:06:26,188 [INFO] General Resource Balance for a in (l, y): adding release(a, l, y)
2025-10-21 11:06:26,189 [INFO] ✔ Completed in 0.00010585784912109375 seconds
2025-10-21 11:06:26,189 [INFO] Binding release in domain (a, l, q)
2025-10

In [4]:
m.release.primary_type

energia.components.commodity.resource.Resource

In [5]:
m.a.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [6]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [7]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-21 11:06:35,472 [INFO] General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
2025-10-21 11:06:35,473 [INFO] ✔ Completed in 0.0001850128173828125 seconds
2025-10-21 11:06:35,473 [INFO] Binding consume in domain (solar, l, q)
2025-10-21 11:06:35,473 [INFO] ✔ Completed in 0.000118255615234375 seconds
2025-10-21 11:06:35,474 [INFO] General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
2025-10-21 11:06:35,474 [INFO] ✔ Completed in 9.942054748535156e-05 seconds
2025-10-21 11:06:35,475 [INFO] Binding consume in domain (wind, l, y)
2025-10-21 11:06:35,475 [INFO] ✔ Completed in 0.00010323524475097656 seconds
2025-10-21 11:06:35,475 [INFO] General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
2025-10-21 11:06:35,476 [INFO] ✔ Completed in 0.00014400482177734375 seconds
2025-10-21 11:06:35,476 [INFO] Binding release in domain (power, l, q)
2025-10-21 11:06:35

## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [ ]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-20 16:30:32,191 [INFO] Binding capacity in domain (wf, l, y)
2025-10-20 16:30:32,193 [INFO] ✔ Completed in 0.000347137451171875 seconds
2025-10-20 16:30:32,194 [INFO] Binding capacity in domain (wf, l, y)
2025-10-20 16:30:32,195 [INFO] ✔ Completed in 0.0003540515899658203 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [ ]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-20 16:30:32,210 [INFO] Binding operate in domain (wf, l, q)
2025-10-20 16:30:32,211 [INFO] ✔ Completed in 0.00028061866760253906 seconds
2025-10-20 16:30:32,212 [INFO] Mapping operate: (wf, l, q) → (wf, l, y)
2025-10-20 16:30:32,213 [INFO] ✔ Completed in 0.000s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-20 16:30:32,230 [INFO] Binding capacity in domain (pv, l, y)
2025-10-20 16:30:32,232 [INFO] ✔ Completed in 0.000438690185546875 seconds
2025-10-20 16:30:32,233 [INFO] Binding capacity in domain (pv, l, y)
2025-10-20 16:30:32,234 [INFO] ✔ Completed in 0.0007221698760986328 seconds
2025-10-20 16:30:32,235 [INFO] Binding operate in domain (pv, l, q)
2025-10-20 16:30:32,237 [INFO] ✔ Completed in 0.00043129920959472656 seconds
2025-10-20 16:30:32,239 [INFO] Mapping operate: (pv, l, q) → (pv, l, y)
2025-10-20 16:30:32,239 [INFO] ✔ Completed in 0.000s


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [ ]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-20 16:30:32,249 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-20 16:30:32,250 [INFO] ✔ Completed in 0.0003254413604736328 seconds
2025-10-20 16:30:32,250 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-20 16:30:32,251 [INFO] ✔ Completed in 0.00025200843811035156 seconds
2025-10-20 16:30:32,253 [INFO] General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory(power.lii, l, y)
2025-10-20 16:30:32,256 [INFO] Binding capacity in domain (lii.charge, l, y)
2025-10-20 16:30:32,257 [INFO] ✔ Completed in 0.0001361370086669922 seconds
2025-10-20 16:30:32,258 [INFO] Binding capacity in domain (lii.discharge, l, y)
2025-10-20 16:30:32,259 [INFO] ✔ Completed in 0.0001442432403564453 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [ ]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-20 16:30:32,267 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
2025-10-20 16:30:32,268 [INFO] ✔ Completed in 0.0002474784851074219 seconds
2025-10-20 16:30:32,270 [INFO] General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
2025-10-20 16:30:32,271 [INFO] ✔ Completed in 0.00034236907958984375 seconds
2025-10-20 16:30:32,273 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
2025-10-20 16:30:32,274 [INFO] ✔ Completed in 0.0002949237823486328 seconds
2025-10-20 16:30:32,276 [INFO] General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
2025-10-20 16:30:32,277 [INFO] ✔ Completed in 0.0001697540283203125 seconds
2025-10-20 16:30:32,278 [INFO] Assuming  power.lii inventory capacity is unbounded in (l, y)
2025-10-20 16:30:32,278 [INFO] Assuming inventory of power.lii is bound by inventory capacity in (l, y)
2025-10-20 16:30:32,279 [INFO]

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [ ]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [ ]:
m.usd.spend.opt()

2025-10-20 16:30:32,322 [INFO] Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
2025-10-20 16:30:32,323 [INFO] ✔ Completed in 0.000s
2025-10-20 16:30:32,323 [INFO] Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
2025-10-20 16:30:32,324 [INFO] ✔ Completed in 0.000s
2025-10-20 16:30:32,324 [INFO] Mapping spend: (usd, l, y, capacity, pv) → (usd, l, y)
2025-10-20 16:30:32,325 [INFO] ✔ Completed in 0.000s
2025-10-20 16:30:32,325 [INFO] Mapping spend: (usd, l, y, operate, pv) → (usd, l, y)
2025-10-20 16:30:32,326 [INFO] ✔ Completed in 0.000s
2025-10-20 16:30:32,326 [INFO] Mapping spend: (usd, l, y, invcapacity, power.lii) → (usd, l, y)
2025-10-20 16:30:32,327 [INFO] ✔ Completed in 0.000s
2025-10-20 16:30:32,327 [INFO] Mapping spend: (usd, l, y, inventory, power.lii) → (usd, l, y)
2025-10-20 16:30:32,328 [INFO] ✔ Completed in 0.000s
2025-10-20 16:30:32,328 [INFO] Generating Program(example2).mps
2025-10-20 16:30:32,337 [INFO] Creating gurobi model for Program(example2)


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros


2025-10-20 16:30:32,344 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-20 16:30:32,361 [INFO] Solution found. Use .output() to display it
2025-10-20 16:30:32,363 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [ ]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [ ]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [ ]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [ ]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>